In [12]:
# Data investigation
import os
import pandas as pd

In [13]:
from fastai import *

In [4]:
os.chdir("/Users/sucooper/Documents/brown-bag/clothing-classifier/clothing_classification")
os.listdir()

['data_exploration.ipynb',
 '.DS_Store',
 'export.pkl',
 'multi_labels.csv',
 'prediction_evaluation.ipynb',
 'apparel-dataset',
 'apparel-dataset.zip',
 '.gitignore',
 'Model_training.ipynb',
 '.git',
 'multilabel_subset.csv']

In [5]:
# Load dataset
if 'apparel-dataset' not in os.listdir():
    if 'apparel-dataset.zip' not in os.listdir():
        print('Dataset has not been downloaded. Downloading now from kaggle')
        !kaggle datasets download -d kaiska/apparel-dataset
    # unzip
    !unzip apparel-dataset.zip -d ./apparel-dataset
# move dataset into the correct location
else:
    print("Dataset already loaded")

Dataset already loaded


In [20]:
# Tabulate the kinds of data
df = pd.DataFrame([item.split('_') for item in os.listdir('apparel-dataset')])
df.columns = ['Colour', 'Item']
df['Count'] = [len(os.listdir('apparel-dataset/' + item)) for item in os.listdir('apparel-dataset')]
df['Count'] = df['Count'].astype(int)


In [21]:
shaped_df = df.pivot(index='Colour', columns='Item', values='Count')
shaped_df

Item,dress,hoodie,pants,shirt,shoes,shorts,skirt,suit
Colour,,,,,,,,
black,450,136,870,715,766,328,126,320
blue,502,131,798,741,523,299,126,136
brown,117,188,311,127,464,127,137,133
green,119,118,227,230,455,135,114,243
pink,136,347,246,132,144,137,513,137
red,800,349,308,332,610,133,130,130
silver,133,125,119,123,403,126,361,127
white,818,138,274,133,600,120,132,354
yellow,566,131,127,139,145,195,409,143


In [24]:
from fastbook import get_image_files

In [46]:
n_per_cat = 10
subset = [{"filename": 'apparel-dataset/' + parent + '/' + filename, "labels": parent } for parent in os.listdir(Path('apparel-dataset')) for filename in os.listdir('apparel-dataset/' + parent)[0:n_per_cat]]

In [47]:
pd.DataFrame(subset).to_csv("labels_subset_10.csv", index=False)

In [16]:
present = [{"Colour": row["Colour"], "Item": row["Item"]} for row in df.iloc()]
all_combinations = [{"Colour": i, "Item": j} for i in df['Colour'].unique() for j in df['Item'].unique()]

difference = [row for row in all_combinations if row not in present]

In [19]:
# These are the ones to get images for
items_to_obtain = [row['Colour'] + '_' + row['Item'] for row in difference]
len(items_to_obtain)


# obtain images for each of these

35

In [52]:
shaped_df

Item,dress,hoodie,pants,shirt,shoes,shorts,skirt,suit
Colour,,,,,,,,
black,450.0,0.0,870.0,715.0,766.0,328.0,0.0,320.0
blue,502.0,0.0,798.0,741.0,523.0,299.0,0.0,0.0
brown,0.0,188.0,311.0,0.0,464.0,0.0,0.0,0.0
green,0.0,0.0,227.0,230.0,455.0,135.0,0.0,243.0
pink,0.0,347.0,246.0,0.0,0.0,0.0,513.0,0.0
red,800.0,349.0,308.0,332.0,610.0,0.0,0.0,0.0
silver,0.0,0.0,0.0,0.0,403.0,0.0,361.0,0.0
white,818.0,0.0,274.0,0.0,600.0,120.0,0.0,354.0
yellow,566.0,0.0,0.0,0.0,0.0,195.0,409.0,0.0


In [60]:
!pip install azure-cognitiveservices-search-imagesearch
!pip install graphviz

In [72]:
from fastai.vision.all import *
from fastai.vision.widgets import ImageClassifierCleaner
from azure.cognitiveservices.search.imagesearch import ImageSearchClient as api
from msrest.authentication import CognitiveServicesCredentials as auth
from pathlib import Path

def search_images_bing(key, term, min_sz=128):
    client = api('https://api.bing.microsoft.com', auth(key))
    return L(client.images.search(query=term, count=150, min_height=min_sz, min_width=min_sz).value)

In [1]:
# search for additional images in bing
from fastbook import search_images_bing, download_images
from pathlib import Path

In [2]:
key = os.environ.get('AZURE_SEARCH_KEY', 'e74445748f594211a2cd25e2abd455ce')
path = Path('collected_data')

In [5]:
results = search_images_bing(key, "green_skirt")


In [7]:
dest = "collected_data/green_skirt"
download_images(dest, urls=results.attrgot('contentUrl'))

In [18]:

for o in items_to_obtain[1:]:
    dest = (path/o)
    if not dest.exists():
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, o)
        download_images(dest, urls=results.attrgot('contentUrl'))


 Download of http://www.bridalshoesuk.co.uk/product_images/r/384/LUXEFUSCH_PAIR__85539_zoom.jpg has failed after 5 retries
 Fix the download manually:
$ mkdir -p collected_data/pink_shoes
$ cd collected_data/pink_shoes
$ wget -c http://www.bridalshoesuk.co.uk/product_images/r/384/LUXEFUSCH_PAIR__85539_zoom.jpg
$ tar xf LUXEFUSCH_PAIR__85539_zoom.jpg
 And re-run your code once the download is successful



In [73]:
results = search_images_bing(key, "green_skirt")

ErrorResponseException: Operation returned an invalid status code 'Resource Not Found'

In [20]:
# convert to multilabel format
dataset_dir = Path('apparel-dataset')
# create a csv file with | filelocation | labels |
fns = get_image_files(dataset_dir)


In [38]:
labels = [{"fname":image, "labels":image.parent.name.replace('_', ' ')} for image in fns]
csv_labels = pd.DataFrame(labels)
csv_labels.to_csv('multi_labels.csv', index=False)

In [63]:
# Create a sub dataset
subset = [{"fname": 'apparel-dataset/' + cat +'/' + i, "labels":cat.replace('_', ' ')} for cat in os.listdir('apparel-dataset') for i in os.listdir('apparel-dataset/' + cat)[0:10]]
subset_df = pd.DataFrame(subset)
subset_df.to_csv('multilabel_subset.csv', index=False)

In [66]:
# Image Block and Category Block
filename = 'multilabel_subset.csv'
multilables = pd.read_csv(filename)

# Turning it into a multilabel data block
path = Path('.')
def get_x(r): return path/r['fname']
def get_y(r): return r['labels'].split(' ')
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock), get_x = get_x, get_y = get_y, item_tfms=RandomResizedCrop(128, min_scale=0.35))
dsets = dblock.datasets(multilables)

In [67]:
dsets

(#370) [(PILImage mode=RGB size=256x256, TensorMultiCategory([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])),(PILImage mode=RGB size=256x256, TensorMultiCategory([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])),(PILImage mode=RGB size=256x256, TensorMultiCategory([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])),(PILImage mode=RGB size=256x256, TensorMultiCategory([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])),(PILImage mode=RGB size=256x256, TensorMultiCategory([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])),(PILImage mode=RGB size=256x256, TensorMultiCategory([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])),(PILImage mode=RGB size=256x256, TensorMultiCategory([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])),(PILImage mode=RGB size=256x256, TensorMultiCategory([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])),

In [43]:
len(dsets.train),len(dsets.valid)

(12936, 3234)

In [32]:
# Multilabel training
clothes = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(460, min_scale=0.5),
    batch_tfms=aug_transforms(size=224, min_scale=0.7))